I created this program as one of my class projects for Clemson University's CPSC 4770 Distributed/Cluster Computing. It runs with Hadoop using Google Trace Data's job events files for the first part of the project and with the task usage files for the second part.

The first part works, however the second part I have to go through and figure out what is wrong. It is on my to-do list.

In [17]:
!cypress-kinit
!klist

Ticket cache: FILE:/home/khelfte/.krb5cc
Default principal: khelfte@PALMETTO.CLEMSON.EDU

Valid starting       Expires              Service principal
12/06/2017 12:14:36  12/13/2017 12:14:36  krbtgt/PALMETTO.CLEMSON.EDU@PALMETTO.CLEMSON.EDU


The following two files, mapper.py and reducer.py, take data from the job_events folder and filter out the jobId, time, and submission status from the files, calculates the total time from submitted (1) to either EVICT (2), FAIL (3), FINISH (4), KILL (5), or LOST (6). It then prints out the top twenty highest times.

In [31]:
%%writefile mapper.py
#!/usr/bin/env /software/anaconda3/2.5.0/bin/python
import sys      

for file in sys.stdin:
    line = file.split(",")
    if (line[3]=='0' or line[3]=='2' or line[3]=='3' or line[3]=='4' or line[3]=='5' or line[3]=='6'):
        print(line[2], line[0], line[3])

Overwriting mapper.py


In [32]:
%%writefile reducer.py
#!/usr/bin/env /software/anaconda3/2.5.0/bin/python
import sys
import numpy
from operator import itemgetter

status = {2: 'EVICT', 3: 'FAIL', 4:'FINISH', 5: 'KILL', 6: 'LOST' }
intermittent = {}
created = ''
id1='0';id2='0';time1='0';time2='0';stat2 = '0'
for file in sys.stdin:
    ids,time,stat = file.split(" ")
    if int(stat) == 0:
        id1 = ids
        time1 = time
    else: 
        id2 = ids
        time2 = time
        stat2 = stat
        if id1 == id2:
            tottime = int(time2) - int(time1)
            intermittent[id1] = (tottime,status[int(stat2)])
inter = 0
for x in sorted(intermittent, key=lambda x: intermittent.get(x)[0], reverse=True):
    print(repr(int(x))+"\t"+repr(intermittent[x][0])+"\t"+repr(intermittent[x][1]))
    inter+=1
    if inter == 20:
        break

Overwriting reducer.py


In [37]:
!hdfs dfs -put reducer.py job_events

In [25]:
!hdfs dfs -cat job_events/job_events/* \
    2>/dev/null \
    | python ./mapper.py \
    | sort \
    | python ./reducer.py

6217291782	2491521082981	'KILL'
4867841244	2488059970906	'KILL'
5782542009	2463114365081	'KILL'
6035592839	2421933921466	'FAIL'
6182354041	2340306707627	'KILL'
6182354078	2340306707617	'KILL'
6182353820	2340304587870	'KILL'
6182354160	2340302039296	'KILL'
6182354117	2340302039286	'KILL'
6182353894	2340302038692	'KILL'
6182353971	2340300639012	'KILL'
6182354009	2340300639004	'KILL'
6182353936	2340300638994	'KILL'
6182353704	2340299854097	'KILL'
6182353777	2340299181750	'KILL'
6182353639	2340298451442	'KILL'
6182353667	2340298451053	'KILL'
6182353613	2340297782400	'KILL'
6182353860	2340297020638	'KILL'
6182366148	2340297020143	'KILL'


In [38]:
!hdfs dfs -rmdir output-0
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar -numReduceTasks 1 \
    -input job_events/job_events/* \
    -output output-0 \
    -file ./mapper.py \
    -mapper mapper.py \
    -file ./reducer.py \
    -reducer reducer.py \

17/12/02 17:39:24 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./mapper.py, ./reducer.py] [/usr/hdp/2.6.0.3-8/hadoop-mapreduce/hadoop-streaming-2.7.3.2.6.0.3-8.jar] /hadoop_java_io_tmpdir/streamjob1022179866488349130.jar tmpDir=null
17/12/02 17:39:26 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/02 17:39:26 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/02 17:39:26 INFO hdfs.DFSClient: Created HDFS_DELEGATION_TOKEN token 17689 for khelfte on ha-hdfs:dsci
17/12/02 17:39:26 INFO security.TokenCache: Got dt for hdfs://dsci; Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, Ident: (HDFS_DELEGATION_TOKEN token 17689 for khelfte)
17/12/02 17:39:27 INFO lzo.GPLNativeCodeLoader: Loaded native gpl library
17/12/02 17:39:27 INFO lzo.LzoCodec: Successfully loaded & initialized nativ

In [42]:
!hdfs dfs -cat output-0/part-00000

6217291782	2491521082981	'KILL'
4867841244	2488059970906	'KILL'
5782542009	2463114365081	'KILL'
6035592839	2421933921466	'FAIL'
6182354041	2340306707627	'KILL'
6182354078	2340306707617	'KILL'
6182353820	2340304587870	'KILL'
6182354160	2340302039296	'KILL'
6182354117	2340302039286	'KILL'
6182353894	2340302038692	'KILL'
6182353971	2340300639012	'KILL'
6182354009	2340300639004	'KILL'
6182353936	2340300638994	'KILL'
6182353704	2340299854097	'KILL'
6182353777	2340299181750	'KILL'
6182353639	2340298451442	'KILL'
6182353667	2340298451053	'KILL'
6182353613	2340297782400	'KILL'
6182353860	2340297020638	'KILL'
6182366148	2340297020143	'KILL'


This code is for the second part of the project, taking the jobId, total CPU time, and number of computing tasks for each unique job. it then prints out the top twenty Ids with the highest total CPU time and another top twenty list of the id with the largest number of computing tasks


TAKE THE TIME TO SPLIT THE TWO PARTS APART SO THAT THEY RUN PROPERLY.

In [25]:
%%writefile part3map.py
#!/usr/bin/env /software/anaconda3/2.5.0/bin/python
import sys

#important things: line2 == jobid line3 is task index line5 is cpu usage
storage = {}
curid = '0'
curtask = '0'
curusage = '0'
for file in sys.stdin:
    line = file.split(",")
    if len(line) == 20: 
        if curid == line[2]: # if job id equals last job id, as it is ordered?
            curtask = int(line[3]) #task index, then store current task
            curusage += float(line[5]) #cpu usage // add CPU usage
        else:
            print(curid, curtask, curusage)# print out the old id
            curid = line[2] #job id, reset values for the next id
            curtask = int(line[3]) #task index
            curusage = float(line[5]) #cpu usage
        

Overwriting part3map.py


In [26]:
%%writefile part3red.py
#!/usr/bin/env /software/anaconda3/2.5.0/bin/python
import sys
curid = ''
curtask=0
curusage=0.0
ids = ''
task=0
usage=0

storage = {}

for file in sys.stdin:
    ids, task,usage = file.split(" ")
    storage[curid] = (curtask, curusage) # else store the previous value and update all currents
inter = 0
for x in sorted(storage, key=lambda x: storage.get(x)[0], reverse=True):
    print(repr(int(x))+"\t"+repr(storage[x][0]))
    inter+=1
    if inter == 20:
        break
inter = 0
print("\n")
for x in sorted(storage, key=lambda x: storage.get(x)[1], reverse=True):
    print(repr(int(x))+"\t"+repr(storage[x][1]))
    inter+=1
    if inter == 20:
        break

Overwriting part3red.py


In [ ]:
!hdfs dfs -cat task_usage/task_usage/* \
    2>/dev/null \
    | python ./part3map.py \
    | sort #\
    #| python ./part3red.py

In [27]:
!hdfs dfs -rm task_usage/part3map.py
!hdfs dfs -rm task_usage/part3red.py
!hdfs dfs -put part3map.py task_usage
!hdfs dfs -put part3red.py task_usage

17/12/06 12:38:21 INFO fs.TrashPolicyDefault: Moved: 'hdfs://dsci/user/khelfte/task_usage/part3map.py' to trash at: hdfs://dsci/user/khelfte/.Trash/Current/user/khelfte/task_usage/part3map.py1512581901594
17/12/06 12:38:40 INFO fs.TrashPolicyDefault: Moved: 'hdfs://dsci/user/khelfte/task_usage/part3red.py' to trash at: hdfs://dsci/user/khelfte/.Trash/Current/user/khelfte/task_usage/part3red.py1512581920789


In [42]:
!hdfs dfs -rmdir output-2
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar -numReduceTasks 1 \
    -input task_usage/task_usage/* \
    -output output-2 \
    -file ./part3map.py \
    -mapper part3map.py #\
    #-file ./part3red.py \
   # -reducer part3red.py \

17/12/06 13:13:53 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./part3map.py] [/usr/hdp/2.6.0.3-8/hadoop-mapreduce/hadoop-streaming-2.7.3.2.6.0.3-8.jar] /hadoop_java_io_tmpdir/streamjob7405211873126493645.jar tmpDir=null
17/12/06 13:14:03 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/06 13:14:04 INFO client.AHSProxy: Connecting to Application History server at dscim003.palmetto.clemson.edu/10.125.8.215:10200
17/12/06 13:14:06 INFO hdfs.DFSClient: Created HDFS_DELEGATION_TOKEN token 17821 for khelfte on ha-hdfs:dsci
17/12/06 13:14:06 INFO security.TokenCache: Got dt for hdfs://dsci; Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, Ident: (HDFS_DELEGATION_TOKEN token 17821 for khelfte)
17/12/06 13:14:09 INFO lzo.GPLNativeCodeLoader: Loaded native gpl library
17/12/06 13:14:09 INFO lzo.LzoCodec: Successfully loaded & initialized native-lzo librar

17/12/06 13:15:32 INFO mapreduce.Job:  map 80% reduce 26%
17/12/06 13:15:33 INFO mapreduce.Job:  map 82% reduce 26%
17/12/06 13:15:34 INFO mapreduce.Job:  map 83% reduce 28%
17/12/06 13:15:35 INFO mapreduce.Job:  map 84% reduce 28%
17/12/06 13:15:36 INFO mapreduce.Job:  map 86% reduce 28%
17/12/06 13:15:37 INFO mapreduce.Job:  map 87% reduce 29%
17/12/06 13:15:38 INFO mapreduce.Job:  map 88% reduce 29%
17/12/06 13:15:39 INFO mapreduce.Job:  map 89% reduce 29%
17/12/06 13:15:40 INFO mapreduce.Job:  map 90% reduce 30%
17/12/06 13:15:49 INFO mapreduce.Job:  map 91% reduce 30%
17/12/06 13:16:14 INFO mapreduce.Job:  map 92% reduce 30%
17/12/06 13:16:16 INFO mapreduce.Job:  map 92% reduce 31%
17/12/06 13:16:39 INFO mapreduce.Job:  map 93% reduce 31%
17/12/06 13:17:05 INFO mapreduce.Job:  map 94% reduce 31%
17/12/06 13:17:29 INFO mapreduce.Job:  map 95% reduce 31%
17/12/06 13:17:32 INFO mapreduce.Job:  map 95% reduce 32%
17/12/06 13:17:56 INFO mapreduce.Job:  map 96% reduce 32%
17/12/06 13:18

In [44]:
!hdfs dfs -cat output-2/part-00000

0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	
0 0 0	

1005190908 9 0.016481	
1005190908 9 0.016481	
1005190908 9 0.016482	
1005190908 9 0.016482	
1005190908 9 0.016482	
1005190908 9 0.016482	
1005190908 9 0.016482000000000004	
1005190908 9 0.016482000000000004	
1005190908 9 0.016482999999999998	
1005190908 9 0.016483	
1005190908 9 0.016483	
1005190908 9 0.016483	
1005190908 9 0.016483	
1005190908 9 0.016483	
1005190908 9 0.016483	
1005190908 9 0.016484	
1005190908 9 0.016484	
1005190908 9 0.016484	
1005190908 9 0.016484000000000002	
1005190908 9 0.016485	
1005190908 9 0.016485	
1005190908 9 0.016485	
1005190908 9 0.016485	
1005190908 9 0.016486	
1005190908 9 0.016486999999999998	
1005190908 9 0.016486999999999998	
1005190908 9 0.016487	
1005190908 9 0.016487	
1005190908 9 0.016488	
1005190908 9 0.016489	
1005190908 9 0.016489	
1005190908 9 0.016489999999999998	
1005190908 9 0.016489999999999998	
1005190908 9 0.016489999999999998	
1005190908 9 0.01649	
1005190908 9 0.016491	
1005190908 9 0.016491	
1005190908 9 0.016491000000000002	
1005190

1005190908 9 0.016981	
1005190908 9 0.016981	
1005190908 9 0.016981000000000003	
1005190908 9 0.016981999999999997	
1005190908 9 0.016981999999999997	
1005190908 9 0.016981999999999997	
1005190908 9 0.016981999999999997	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982	
1005190908 9 0.016982999999999998	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016983	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984	
1005190908 9 0.016984000000000003	
1005190908 9 0.016984000000000003	
1005190908 9 0.016984999999999

1005190908 9 0.018337999999999997	
1005190908 9 0.018342	
1005190908 9 0.018347000000000002	
1005190908 9 0.018352	
1005190908 9 0.018361	
1005190908 9 0.018369	
1005190908 9 0.018369000000000003	
1005190908 9 0.018369000000000003	
1005190908 9 0.018374	
1005190908 9 0.018375999999999997	
1005190908 9 0.018384	
1005190908 9 0.018385	
1005190908 9 0.018385	
1005190908 9 0.018393	
1005190908 9 0.018406	
1005190908 9 0.018408	
1005190908 9 0.018410000000000003	
1005190908 9 0.018415	
1005190908 9 0.018420999999999996	
1005190908 9 0.018437000000000002	
1005190908 9 0.018447	
1005190908 9 0.018453	
1005190908 9 0.018459	
1005190908 9 0.018473	
1005190908 9 0.018484999999999998	
1005190908 9 0.018492	
1005190908 9 0.018498999999999998	
1005190908 9 0.018501	
1005190908 9 0.018503000000000002	
1005190908 9 0.018508999999999998	
1005190908 9 0.018516	
1005190908 9 0.018517999999999996	
1005190908 9 0.018525	
1005190908 9 0.018532999999999997	
1005190908 9 0.018547	
1005190908 9 0.018549	
1005

1005246041 4 0.008716	
1005246041 4 0.008719	
1005246041 4 0.008721	
1005246041 4 0.008723999999999999	
1005246041 4 0.008724	
1005246041 4 0.008728	
1005246041 4 0.008731	
1005246041 4 0.008735	
1005246041 4 0.008735	
1005246041 4 0.008735	
1005246041 4 0.008742999999999999	
1005246041 4 0.008742999999999999	
1005246041 4 0.008743	
1005246041 4 0.008745	
1005246041 4 0.008745	
1005246041 4 0.00875	
1005246041 4 0.008755	
1005246041 4 0.008756	
1005246041 4 0.008762	
1005246041 4 0.008763	
1005246041 4 0.008764000000000001	
1005246041 4 0.008766	
1005246041 4 0.008768	
1005246041 4 0.008773999999999999	
1005246041 4 0.008778	
1005246041 4 0.008785000000000001	
1005246041 4 0.008789	
1005246041 4 0.008789	
1005246041 4 0.008789	
1005246041 4 0.00879	
1005246041 4 0.008795999999999998	
1005246041 4 0.008796	
1005246041 4 0.008796	
1005246041 4 0.008796	
1005246041 4 0.008796	
1005246041 4 0.008797	
1005246041 4 0.008797	
1005246041 4 0.008797	
1005246041 4 0.008797	
1005246041 4 0.008804

...

1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522

1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522

1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522

1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522

1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522 0 0.0	
1161381522